In [14]:
import math # for log
import random # for vignere key generation

In [15]:
DISTINCT = 37
S_ID = 2 # Self Id
R_ID = 1 # Retrieval sender Id
CHECK = ord('-')-ord('a') # for '-'
FIX = 26 # for '-'
FIX_NUM = 27 # for number '0' and so on...

In [16]:
RCVD_MSG = open('send_over_unsafe_network.txt', 'r').read()
RCVD_MSG

'ojw12nwf7y3io2i0th3gm62emi29bqc0wqfy928ed6r3g9m1dz2cq-t677nvfiqqwvfkvyb33ielfikl2lghmve2xr5od-1ydljv02unyx1u5drhlhforec7cke9oeebe8qoiszy61s8d6adxiat0f0nrqf651bv4q0xpluw4upaawfxyssfszujmztt52uvpy-cbesufa0olfpiy3pms6k9925-7pwkvvmvhxiyx438qfym3zhmg7xog0mih27l7ehm7pxums474tyoobj-k20l-sqs73cfrrpd6rlsoc400lh0z7svmqdfusnovt841q82ulck3s7goalz94pj55oiu0hx5sor61sydnaewnap4zr2s7duabomcrnsmt5jb5oy7i8qv9'

In [17]:
CA_e, CA_n = map(int, open('public_CA.txt', 'r').read().split())
CA_e, CA_n

(246345344925610181931857553251542711701424910022313570595677899783215214238902994390590396085980415507144140973070287063830301490629433954278206767301747392071482511942984864014232374847201602916449323384316065975370539707471296419095130060603300030677733964983745736329718419233730942853736233836808696138283,
 300358006975011495139377803157720861039011374118356033513884791180036437575075580917566356096898192054991830212564933735009645927464784728794775803648548625543909631078259511747297330031345132835079465931131268976922480095601661069191595064747281103228433038159412442515438435289649092095381097944090770591873)

In [18]:
S_keys = open('private_' + str(S_ID) + '.txt', 'r').read() # Encrypted by CA
S_keys

'evqb8759bw2ela9rsff65bi52w9j1tu2v0aje0xftzlxop3bvj41826imvmlkagoy7gqfpu-b0crrjm0o9obf0qhlzyt0eehp7c63485lomhnijmnn2mg9evdgmh39p937ow-xl33nsbe5e7zdlig--kpfhn8z9a846wrqj0e69xyx4nsi8ycarkf9py5y79jzvi7hv5hdt93w4q8dy2i-blhgvru0bdp8m5szjg26gpkpjt4ja0rmgnzf4z-funytmx83vnhm0varn3s6ydet0uczwbceolhkgrgnoth8rztqpxqae0leyw1hcyiwe-cwt8c1rk8pmgl5vbt439oszxl5gjoxwoqavjoitw2m8lfh0gsn3ozvtgk5lc0k6e6p15y59jk2cela340lvi91p2k0va47qmbu6v0rz7u8i-kmhtundipg7gv9l5y6jdoay73lrhl6098yq-lxves8nxn97lw093hrq92pjb1mxatjjo4jeu-yrxcqop6tmuw46-48-e5q4gnqfftgm4ycwudat0xm39sz3d-b8l6fq2nq5wzubhj0ycchn1gvxuz7vxjkwky1xb6z7ieqxl1apxu0yunuc5w7v6d1ax748uev5dxf6ojsxpvhdkqwz0qf1cd5zev4cl-xm9b1944pvydd67i728atj7v6t93frz5-5j18dyp4o5s4o-nyjcwnemktwoejokntf7jmulrkisoq9hsmxm6lq62ere2t7evw82jx2euk1plp0zf-n7rij1k9w66lvwdi2w-bpcl9jy2i-bgs5xtm3p6py026l4481dbpv8bzdek4gwh5kith2wumr0rru3wrdt1ktzohnirwuhd-a4z22dqcfggv0ay54cmwk137q9bd-rjdwu2w37f4lt--dfrvsuhq2i5m4qwztjo8t2yflfy4bxe1ipyvgmt4rilaqf3rsn9vjispnb9z9jknomqgp37znhu81x7'

In [19]:
def find_block_size(num_diff_chars, n):
    block_size = int(math.log(n, num_diff_chars))

    if num_diff_chars**block_size >= n:
        return block_size-1
    else:
        return block_size

In [20]:
def decrypt_RSA(ci_text, key, n):
    pl_text = ''
    r = find_block_size(DISTINCT, n)
    s = r+1
    
    for i in range(0, len(ci_text), s):
        block = ci_text[i:i+s]
        C = 0
        for j, ch in enumerate(block[::-1]): # Reverse of encryption
            if ch=='-':
                C += (FIX)*(DISTINCT**j)
            elif '0' <= ch <= '9':
                C += (ord(ch) - ord('0') + FIX_NUM)*(DISTINCT**j)
            else:
                C += (ord(ch) - ord('a'))*(DISTINCT**j)
        M = pow(C, key, n)
    
        updated_block = ''
        for j in range(r-1,-1,-1):
            temp = M // (DISTINCT**j)
            M -= temp*(DISTINCT**j)
            
            # for Numbers
            if FIX_NUM <= temp <= FIX_NUM+9:
                updated_block += chr(temp-FIX_NUM + ord('0'))
            else:
                if temp == FIX: temp = CHECK # for '-'
                updated_block += chr(temp+ord('a'))
            
        pl_text += updated_block[::-1]
    
    return pl_text

In [21]:
def remove_extra_a(with_a):
    return with_a.rstrip('a')

In [22]:
S_e, S_d, S_p, S_q = map(int, list(map(remove_extra_a, decrypt_RSA(S_keys, CA_e, CA_n).split('-'))))
S_n = S_p * S_q
S_e, S_p

(315820632263979017878161762394302849274589930349009512674958537510958416196229174309098013513169878744460359883219367985131777278006572591996609575442911810174626303478154563402971881960543738514710946295281295075523784808614096615851071419616372788891886068112076614158409702857991771153536869252322291962319,
 17672622379042383543544954302902764896965817581893365655977705612195474739267021910105784770178791994465127759310359914277169007754222053366912281484916703)

In [23]:
for row in open('public_dir.txt', 'r').read().split('\n')[:-1]:
    row_details = row.split(' ')
    if int(row_details[0]) == R_ID:
#         R_e, R_n = row_details[1], row_details[2]
        R_keys = row_details[1]
        R_e, R_n = map(int, list(map(remove_extra_a, decrypt_RSA(R_keys, CA_e, CA_n).split('-'))))
R_e, R_n

(189125017868028636027158360011998470834279375927220104042860142027994757870456794678346707433564701876733199586646168603749840328713730068149176297392555866629292298642774128813058833887467862677281693485137536864464139684158052757945273968150505760451029010549580799681444461865842634918020762228809853592631,
 242040887631779606594853804079271825485710532099336836847866212702935771705031238008677150944233780835224865168609296071165478934663144144841441840395522571882199996629949660801960979287161151988874673383609196088337722347350738497735657892399090688105776977351730172656284782700536454965698606234358145110461)

In [24]:
decrypt1_msg = decrypt_RSA(RCVD_MSG, S_d, S_n).rstrip('a')
decrypt1_msg

'd2jgxkw65n0vtn1occziditxej3fn64wcbns5tiobe69u32jgrz0x6lysyx18-zc3wv7o0kqx1yakh5t0q1tx35kz35m-fpmk-podix8i-86m9re4ivae7x1uttscxs95rfz3o36xuln7rryxszdosehkga4dwvvh7lzwzir0tskvq96seqwppy55mp6tm-oxqj9o'

In [25]:
key_vig, encrypt_vig = map(remove_extra_a, decrypt_RSA(decrypt1_msg, R_e, R_n).split('-'))
key_vig, encrypt_vig

('kyomhwctjbzndqilfexv',
 'rczxvpjbbjrneuifymcpvbokadklqbrohuvlhvxuipwplpjxmfzeguxlwxbyklrtlngpnbqrldbswixoydqaykptejqhvmqwqclplcbgtxgk')

In [26]:
def decrypt_vignere(ci_text, key):
    """
        Takes cipher_text(Lowercase Letters) and key as argument and returns decrypted plain_text.
    """
    pl_text = ''
    key_len = len(key)
    for i in range(len(ci_text)):
        pl_char = ( ord(ci_text[i]) - ord('a') - ord(key[i%key_len]) + ord('a') )%26
        pl_text += chr( pl_char + ord('a') )
    
    return pl_text

In [27]:
decrypt_vignere(encrypt_vig, key_vig)

'hellothisisabeautifuldaythishasbeenacrazyridethedeardepartedandhereweareinthreatofcoronaviruswillyoubenumber'